## Imports

In [1]:
import json
import os
from datetime import timedelta, datetime
from time import sleep
from sys import argv
import gensim
import gensim.corpora as corpora
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pyLDAvis
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
import torch_geometric
from functools import reduce
from tqdm import tqdm
from Preprocessing import Preprocessor
preprocessor = Preprocessor(0)
tqdm.pandas()

c:\Python39\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


## Check if data is already joined locally

In [2]:
joined_exists = os.path.exists('khan_joined.csv')
khan = pd.DataFrame([])
if joined_exists:
    khan = pd.read_csv('khan_joined.csv')
khan.head(5)

,course,unit,lesson,video_title,about,transcript,topic,transcript_cleaned,tokens,transcript_n_entries
0,computer,Intro to JS: Drawing & Animation,Intro to programming,What is Programming?,Programming is the process of creating a set o...,"Hi, welcome to programming! If you've never le...",Computing,"hi, welcome programming! never learned program...","['hi,', 'welcome', 'programming!', 'never', 'l...",183
1,computer,Intro to JS: Drawing & Animation,Coloring,The Power of the Docs,Created by Pamela Fox.,Voiceover: Ok so you've\r\nmade a few programs...,Computing,"voiceover: ok made programs, might wondering e...","['voiceover:', 'ok', 'made', 'programs,', 'mig...",542
2,computer,Intro to HTML/CSS: Making webpages,Further learning,HTML validation,Learn how to validate your webpages with the W...,"- [Voiceover] On Khan Academy, we pop up the o...",Computing,"- khan academy, pop oh noes guide tell somethi...","['-', 'khan', 'academy,', 'pop', 'oh', 'noes',...",172
3,computer,Intro to SQL: Querying and managing data,SQL basics,Welcome to SQL,SQL is useful for creating and querying relati...,- [Instructor] The world is full of data. Ever...,Computing,- world full data. every app use full data. kh...,"['-', 'world', 'full', 'data.', 'every', 'app'...",203
4,computer,Intro to SQL: Querying and managing data,SQL basics,S-Q-L or SEQUEL?,How is it pronounced? Why? Let's discuss...,"At this point, you've probably heard me\r\npro...",Computing,"point, probably heard pronounce sql two ways--...","['point,', 'probably', 'heard', 'pronounce', '...",129


## Load each domain separately if not already joined

In [3]:
if not joined_exists:
    computing = pd.read_csv("Datasets\\KhanAcademy\\Computing.csv")
    computing = computing.dropna()
    computing['topic'] = 'Computing'
    computing.info()

In [4]:
if not joined_exists:
    economics = pd.read_csv("Datasets\\KhanAcademy\\Economics.csv")
    economics = economics.dropna()
    economics['topic'] = 'Economics'
    economics.info()

In [5]:
if not joined_exists:
    humanities = pd.read_csv("Datasets\\KhanAcademy\\Humanities.csv")
    humanities = humanities.dropna()
    humanities['topic'] = 'Humanities'
    humanities.info()

In [6]:
if not joined_exists:
    math = pd.read_csv("Datasets\\KhanAcademy\\Math.csv")
    math = math.dropna()
    math['topic'] = 'Math'
    math.info()

In [7]:
if not joined_exists:
    science = pd.read_csv("Datasets\\KhanAcademy\\Science.csv")
    science = science.dropna()
    science['topic'] = 'Science'
    science.info()

## Display the unique courses for each domain

In [8]:
if not joined_exists:
    computing['course'].value_counts()

In [9]:
if not joined_exists:
    economics['course'].value_counts()

In [10]:
if not joined_exists:
    humanities['course'].value_counts()

In [11]:
if not joined_exists:
    math['course'].value_counts()

In [12]:
if not joined_exists:
    science['course'].value_counts()

## Join the domains into one file

In [13]:
if not joined_exists:
    khan_dfs = [computing, economics, humanities, math, science]
    khan = pd.concat(khan_dfs, axis=0)
    khan.info()

## Clean up some of the course so they can be used as ground truth labels

In [14]:
# if not joined_exists:
# remap the courses to more broad categories: https://stackoverflow.com/a/16476974
labels = ['physics', 'chemistry', 'biology', 'algebra', 'geometry', 'statistics', 'calculus', 'history', 'economics', 'computer']
for lbl in labels:
    # print(lbl)
    for index, row in khan.iterrows():
        if lbl in row['course'].lower():
            row['course'] = lbl
khan['course'].value_counts()

algebra                                         1287
calculus                                         947
chemistry                                        909
history                                          769
biology                                          757
physics                                          743
statistics                                       626
economics                                        502
geometry                                         345
Finance and capital markets                      317
Electrical engineering                           198
Pixar in a Box                                   175
US government and civics                         139
Cosmology and astronomy                           89
AP®︎/College US Government and Politics           87
Trigonometry                                      76
computer                                          75
Differential equations                            70
Storytelling                                  

In [15]:
khan.tail(2)

,course,unit,lesson,video_title,about,transcript,topic,transcript_cleaned,tokens,transcript_n_entries
8259,physics,Review for AP Physics 1 exam,AP Physics 1 free response questions 2015,Question 4: 2015 AP Physics 1 free response,Identical spheres falling from the same height...,- [Voiceover] Two identical\r\nspheres are rel...,Science,- two identical spheres released device time e...,"['-', 'two', 'identical', 'spheres', 'released...",673
8260,physics,Review for AP Physics 1 exam,AP Physics 1 free response questions 2015,Question 5: 2015 AP Physics 1 free response,Fundamental frequencies (first harmonics) of s...,- [Voiceover] The figure\r\nabove shows a stri...,Science,- figure shows string one end attached oscilla...,"['-', 'figure', 'shows', 'string', 'one', 'end...",901


## Clean up the transcripts using the preprocessor

In [16]:
if not joined_exists:
    # It takes 21 min to run SpaCy preprocessing over each record in the data
    khan['transcript_cleaned'] = khan['transcript'].progress_apply(lambda x: preprocessor.clean(x, fast=True))
    print(khan['transcript_cleaned'][0])

## Tokenize the transcript using the preprocessor

In [17]:
if not joined_exists:
    # clear up the lists to be unique only.
    khan['tokens'] = khan['transcript'].progress_apply(lambda x: preprocessor.clean(x, tokenize=True, fast=True))
    print(len(khan['tokens'][0]))
    print(khan['tokens'][0])

In [18]:
if not joined_exists:
# if not False:
    khan['transcript_n_entries'] = khan['tokens'].progress_apply(lambda x: len(x))
    khan.head(5)

In [19]:
khan.head(1)

,course,unit,lesson,video_title,about,transcript,topic,transcript_cleaned,tokens,transcript_n_entries
0,computer,Intro to JS: Drawing & Animation,Intro to programming,What is Programming?,Programming is the process of creating a set o...,"Hi, welcome to programming! If you've never le...",Computing,"hi, welcome programming! never learned program...","['hi,', 'welcome', 'programming!', 'never', 'l...",183


## Save the results to a local file to speed up rerunning further documents

In [20]:
if not joined_exists:
# if not False:
    khan.to_csv("khan_joined.csv", index=False)

## Train a simple unsupervised LDA model on the transcripts

In [21]:
num_topics = len(khan['course'].value_counts())
print(num_topics)

22


In [22]:
print(type(khan['tokens'][0]))

<class 'str'>


In [23]:
%%time
# LDA Normal
# temp = [d.split() for d in corpus]
temp = [doc.split() for doc in khan['tokens']]
print(type(temp))
words = corpora.Dictionary(temp)
words.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
corpus = [words.doc2bow(doc) for doc in temp]

<class 'list'>
CPU times: total: 4.34 s
Wall time: 4.34 s


In [24]:
%%time
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=words,
                                           num_topics=num_topics,
                                           random_state=2,
                                           update_every=1,
                                           passes=15,
                                           alpha='auto')

CPU times: total: 1min 50s
Wall time: 1min 44s


## Prepare a visualization of the LDA model for investigation

In [25]:
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, words, mds='mmds', R=10)

c:\Python39\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


In [26]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
21     0.046855  0.352133       1        1  10.587495
15    -0.203032 -0.043992       2        1   9.230212
18    -0.079421  0.346920       3        1   8.912244
2     -0.081495  0.007397       4        1   7.694979
1     -0.340482 -0.145893       5        1   6.263166
16     0.196006 -0.313863       6        1   6.135380
17     0.067137  0.114214       7        1   5.556487
3      0.154371  0.357653       8        1   5.399918
4     -0.170629 -0.244389       9        1   4.857525
14    -0.288003  0.216200      10        1   4.719675
6     -0.083157  0.225440      11        1   4.687601
10    -0.383329  0.099563      12        1   4.381133
5      0.144060 -0.140873      13        1   2.723212
12     0.363959  0.026880      14        1   2.645609
8      0.382587 -0.203900      15        1   2.631814
7      0.332774  0.116491      16        1   2.493057
19     0.238906  0.270542      17        1   2.453210
9      0.175689 -0.013008      18        1   2.400632
11    -0.360694 -0.040432      19        1   2.220980
20     0.042640 -0.193278      20        1   2.131307
0     -0.183470 -0.426774      21        1   1.117528
13     0.028727 -0.367032      22        1   0.756837, topic_info=             Term          Freq         Total Category  logprob  loglift
314          'x',  28332.000000  28332.000000  Default  10.0000  10.0000
595      'times',  37100.000000  37100.000000  Default   9.0000   9.0000
2862  'negative',  26597.000000  26597.000000  Default   8.0000   8.0000
2549    'energy',  11465.000000  11465.000000  Default   7.0000   7.0000
667       'plus',  24112.000000  24112.000000  Default   6.0000   6.0000
...           ...           ...           ...      ...      ...      ...
5055    'bright',    227.898290    257.294896  Topic22  -4.8135   4.7625
1798      'path',    254.106364    801.142098  Topic22  -4.7046   3.7355
1299     'speed',    281.722428   1894.968058  Topic22  -4.6015   2.9777
46         'got',    300.570247   6758.566860  Topic22  -4.5367   1.7709
1577    'length',    276.787831   3435.869209  Topic22  -4.6191   2.3650

[544 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
7765      10  0.998179     '$100',
104        2  0.334948        '-',
104        5  0.161758        '-',
104       10  0.000131        '-',
104       12  0.502093        '-',
...      ...       ...         ...
965       16  0.005319     'zero',
965       18  0.000367     'zero',
965       20  0.028981     'zero',
965       22  0.020176     'zero',
12793     12  0.997288  'zucker:',

[2224 rows x 3 columns], R=10, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[22, 16, 19, 3, 2, 17, 18, 4, 5, 15, 7, 11, 6, 13, 9, 8, 20, 10, 12, 21, 1, 14])

## Run a simple prediction as a test

In [28]:
new_text_corpus = words.doc2bow(khan['transcript'][0].split())
print(len(new_text_corpus))
prediction = lda_model.get_document_topics(new_text_corpus)
prediction.sort(key = lambda x: x[1], reverse = True)
print(prediction)
prediction = prediction[0][0]
print("Predicted Topic: %d" % prediction)
lda_model.show_topic(prediction)

0
[(2, 0.098402314), (21, 0.0809694), (18, 0.075872764), (17, 0.073218636), (15, 0.06998841), (3, 0.06832001), (5, 0.056444004), (4, 0.053440616), (6, 0.04724776), (12, 0.042431854), (9, 0.040600665), (19, 0.03880761), (11, 0.03551189), (20, 0.03448599), (7, 0.03177102), (16, 0.028411347), (1, 0.02583002), (0, 0.0225086), (14, 0.021515336), (10, 0.02058558), (13, 0.018438345), (8, 0.015197829)]
Predicted Topic: 2


[("'maybe',", 0.02533247),
 ("'kind',", 0.01516013),
 ("'say,',", 0.012839129),
 ("'things',", 0.0116827525),
 ("'lot',", 0.009117131),
 ("'is,',", 0.008418298),
 ("'know,',", 0.008062178),
 ("'whole',", 0.007447749),
 ("'essentially',", 0.0072026076),
 ("'it,',", 0.006912621)]